In [39]:
import tensorflow as tf
batch_size = 64
(x_train, Y_train), (x_test, Y_test) = tf.keras.datasets.cifar10.load_data()

In [8]:
import numpy as np
all_digits = np.concatenate([x_train, x_test])
all_output = np.concatenate([Y_train, Y_test])
all_output_one_hot = tf.one_hot(np.squeeze(all_output), 10)
all_digits = all_digits.astype("float32") / 255.0

In [9]:
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_output_one_hot))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.batch(batch_size).prefetch(1)

In [10]:
DATASET_SIZE=60000
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = dataset
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [149]:
for line in dataset.take(1):
    print(line[1])

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]], shape=(10, 10), dtype=float32)


In [11]:
model_16 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(32,32,3)),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
    ], name = "block_1"
)

model_32 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(32,32,16)),
        tf.keras.layers.Conv2D(32, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
    ], name = "block_2"
)

model_64 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(16,16,32)),
        tf.keras.layers.Conv2D(64, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(8,8), strides=(1, 1), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax")
    ], name = "block_3"
)

In [12]:
class ForcedNet20(tf.keras.Model):
    def __init__(self, block_1, block_2, block_3):
        super(ForcedNet20, self).__init__()
        self.block_1 = block_1
        self.output_1 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(32,32), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(10, activation="softmax")
            ]
        )
        
        self.block_2 = block_2
        self.output_2 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(16,16), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(10, activation="softmax")
            ]
        )
        
        self.block_3 = block_3
    def compile(self, optimizer, loss_fn):
        super(ForcedNet20, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
    
    def train_step(self, data):
        images = data[0]
        labels = data[1]
        
        with tf.GradientTape(persistent=True) as tape:
            x = self.block_1(images)
            predictions_1 = self.output_1(x)
            
            x = self.block_2(x)
            predictions_2 = self.output_2(x)

            predictions_3 = self.block_3(x)
            
            loss_1 = self.loss_fn(labels, predictions_1)
            loss_2 = self.loss_fn(labels, predictions_2)
            loss_3 = self.loss_fn(labels, predictions_3)
        
        grads_1 = tape.gradient(loss_1, self.block_1.trainable_weights)
        grads_2 = tape.gradient(loss_2, self.block_2.trainable_weights)
        grads_final = tape.gradient(loss_3, self.block_3.trainable_weights)
        
        grads_1_output = tape.gradient(loss_1, self.output_1.trainable_weights)
        grads_2_output = tape.gradient(loss_2, self.output_2.trainable_weights)
                                
        self.optimizer.apply_gradients(
            zip(grads_1, self.block_1.trainable_weights,)
        )
        self.optimizer.apply_gradients(
            zip(grads_1_output, self.output_1.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2, self.block_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2_output, self.output_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_final, self.block_3.trainable_weights)
        )
        
        return {"Block_1_Loss": loss_1, "Block_2_Loss": loss_2, "Block_3_Loss": loss_3}

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
class Metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'mse': mean_squared_error(y_val, y_predict),
        })
        self._data.append({
            'mae': mean_absolute_error(y_val, y_predict),
        })
        self._data.append({
            'mape': mean_absolute_percentage_error(y_val, y_predict),
        })
        self._data.append({
            'accuracy': accuracy_score(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data

metrics = Metrics()

ForcedLearner = ForcedNet20(model_16, model_32, model_64)
ForcedLearner.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=tf.keras.losses.CategoricalCrossentropy(),
)

ForcedLearner.fit(train_dataset, epochs=10, validation_data=(val_dataset))
metrics.get_data()

Epoch 1/10
6000/6000 [==============================] - 37s 6ms/step - BLock_1_Loss: 1.3077 - Block_2_Loss: 1.2956 - Block_3_Loss: 1.2272
Epoch 2/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 1.1346 - Block_2_Loss: 1.0871 - Block_3_Loss: 1.0623
Epoch 3/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 1.0885 - Block_2_Loss: 1.0205 - Block_3_Loss: 1.0034
Epoch 4/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 1.0620 - Block_2_Loss: 0.9872 - Block_3_Loss: 0.9765
Epoch 5/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 1.0388 - Block_2_Loss: 0.9561 - Block_3_Loss: 0.9489
Epoch 6/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 1.0164 - Block_2_Loss: 0.9264 - Block_3_Loss: 0.9225
Epoch 7/10
6000/6000 [==============================] - 36s 6ms/step - BLock_1_Loss: 0.9993 - Block_2_Loss: 0.9051 - Block_3_Loss: 0.9017
Epoch 8/10
6000/6000 [============

In [32]:
Resnet = tf.keras.applications.ResNet50(weights=None, input_shape=(32,32,3), classes=10)

Resnet.compile(loss="CategoricalCrossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), metrics=['mse', 'mae', 'mape', 'accuracy'])
history = Resnet.fit(train_dataset, batch_size=64, epochs=10)

Epoch 1/10
2816/6000 [=============>................] - ETA: 1:09 - loss: 2.2238 - mse: 0.0874 - mae: 0.1623 - mape: 81164456.0000 - accuracy: 0.2610

KeyboardInterrupt: 

In [35]:
inputs = tf.keras.Input(shape=(32,32,3))
x = model_16(inputs)
x = model_32(x)
outputs = model_64(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="vanilla_model")

In [37]:
model.summary()

Model: "vanilla_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block_1 (Sequential)        (None, 32, 32, 16)        14368     
                                                                 
 block_2 (Sequential)        (None, 16, 16, 32)        60128     
                                                                 
 block_3 (Sequential)        (None, 10)                240714    
                                                                 
Total params: 315,210
Trainable params: 315,210
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss="CategoricalCrossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), metrics=['mse', 'mae', 'mape', 'accuracy'])
history = model.fit(train_dataset, batch_size=64, epochs=10)

Epoch 1/10
6000/6000 [==============================] - 46s 7ms/step - loss: 1.0716 - mse: 0.0504 - mae: 0.1005 - mape: 50272064.0000 - accuracy: 0.6150
Epoch 2/10
6000/6000 [==============================] - 45s 7ms/step - loss: 1.0402 - mse: 0.0490 - mae: 0.0979 - mape: 48925988.0000 - accuracy: 0.6243
Epoch 3/10
6000/6000 [==============================] - 44s 7ms/step - loss: 1.0104 - mse: 0.0477 - mae: 0.0953 - mape: 47648472.0000 - accuracy: 0.6350
Epoch 4/10
6000/6000 [==============================] - 44s 7ms/step - loss: 0.9863 - mse: 0.0467 - mae: 0.0932 - mape: 46621988.0000 - accuracy: 0.6449
Epoch 5/10
6000/6000 [==============================] - 44s 7ms/step - loss: 0.9665 - mse: 0.0458 - mae: 0.0914 - mape: 45713264.0000 - accuracy: 0.6519
Epoch 6/10
6000/6000 [==============================] - 44s 7ms/step - loss: 0.9508 - mse: 0.0450 - mae: 0.0899 - mape: 44944308.0000 - accuracy: 0.6593
Epoch 7/10
6000/6000 [==============================] - 45s 8ms/step - loss: 0.932